# Build the logbook for SCAN data on new Spectractor

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : June 16th 2021
- Update : July 7th 2021
- Update : September 24th 2021 : to run quickLookExp at CC and locally


Works directory per directory (day by day)

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) at CC
- works with kernel **python 3** locally

In [47]:
import sys
sys.path

['/Users/dagoret/MacOSX/GitHub/LSST/AuxTelComm/notebookscc/ana_summer2021',
 '/users/dagoret/anaconda3/lib/python38.zip',
 '/users/dagoret/anaconda3/lib/python3.8',
 '/users/dagoret/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/aeosa',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/Spectractor-2.1-py3.8.egg',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/coveralls-3.2.0-py3.8.egg',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/astroquery-0.4.4.dev6955-py3.8.egg',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/docopt-0.6.2-py3.8.egg',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/mimeparse-0.1.3-py3.8.egg',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/staralt-0.0.1a0-py3.8.egg',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/rail-0.1.dev0-py3.8.egg',
 '/users/dagoret/anaconda3/lib/python3.8/site-packages/qp-0.5.dev10+gc

In [48]:
#version="v3.0" # previous orientation
version="v4.0"  # new orientation

In [49]:
# reconstructed / good /xpos /ypos

if version == "v3.0":
    info={\

          # with red filter
          'holo4_003_RG610_HD160617_20210707_000234_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000235_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000236_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000237_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000238_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000239_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000240_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000241_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000242_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000243_quickLookExp.fits':(1,1,600.,2100),\
          'holo4_003_RG610_HD160617_20210707_000244_quickLookExp.fits':(1,1,1300.,700),\
          'holo4_003_RG610_HD160617_20210707_000245_quickLookExp.fits':(1,1,250.,1800),\
          'holo4_003_RG610_HD160617_20210707_000246_quickLookExp.fits':(1,1,250.,1800),\
          'holo4_003_RG610_HD160617_20210707_000247_quickLookExp.fits':(1,1,250.,1800),\
      


            # wide scan index 14 ....  21...
          'holo4_003_empty_HD160617_20210707_000248_quickLookExp.fits':(1,1,1400.,700.),\
          'holo4_003_empty_HD160617_20210707_000249_quickLookExp.fits':(1,1,1400.,700.),\
          'holo4_003_empty_HD160617_20210707_000250_quickLookExp.fits':(1,1,400.,1800.),\
          'holo4_003_empty_HD160617_20210707_000251_quickLookExp.fits':(1,1,400.,1800.),\
          'holo4_003_empty_HD160617_20210707_000252_quickLookExp.fits':(1,1,400.,1800.),\
          'holo4_003_empty_HD160617_20210707_000253_quickLookExp.fits':(1,1,0.,800.),\
          'holo4_003_empty_HD160617_20210707_000254_quickLookExp.fits':(1,1,0.,800.),\
          'holo4_003_empty_HD160617_20210707_000255_quickLookExp.fits':(1,1,0.,800.),\
      

            # narrrow scan : index 76 .....96
          'holo4_003_empty_HD160617_20210707_000317_quickLookExp.fits':(1,1,500.,2100.),\
          'holo4_003_empty_HD160617_20210707_000318_quickLookExp.fits':(1,1,500.,2100.),\
          'holo4_003_empty_HD160617_20210707_000319_quickLookExp.fits':(1,1,500.,1800.),\
          'holo4_003_empty_HD160617_20210707_000320_quickLookExp.fits':(1,1,500.,1800.),\
          'holo4_003_empty_HD160617_20210707_000321_quickLookExp.fits':(1,1,500.,1700.),\
          'holo4_003_empty_HD160617_20210707_000322_quickLookExp.fits':(1,1,500.,1600.),\
          'holo4_003_empty_HD160617_20210707_000323_quickLookExp.fits':(1,1,500.,1500.),\
          'holo4_003_empty_HD160617_20210707_000324_quickLookExp.fits':(1,1,400.,2200.),\
          'holo4_003_empty_HD160617_20210707_000325_quickLookExp.fits':(1,1,400.,2100.),\
          'holo4_003_empty_HD160617_20210707_000326_quickLookExp.fits':(1,1,350.,1860.),\
          'holo4_003_empty_HD160617_20210707_000327_quickLookExp.fits':(1,1,400.,1800.),\
          'holo4_003_empty_HD160617_20210707_000328_quickLookExp.fits':(1,1,400.,1750.),\
          'holo4_003_empty_HD160617_20210707_000329_quickLookExp.fits':(1,1,400.,1750.),\
          'holo4_003_empty_HD160617_20210707_000330_quickLookExp.fits':(1,1,400.,1600.),\
          'holo4_003_empty_HD160617_20210707_000331_quickLookExp.fits':(1,1,300.,2200.),\
          'holo4_003_empty_HD160617_20210707_000332_quickLookExp.fits':(1,1,300.,2100.),\
          'holo4_003_empty_HD160617_20210707_000333_quickLookExp.fits':(1,1,350.,2000.),\
          'holo4_003_empty_HD160617_20210707_000334_quickLookExp.fits':(1,1,350.,1800.),\
          'holo4_003_empty_HD160617_20210707_000335_quickLookExp.fits':(1,1,350.,1750.),\
          'holo4_003_empty_HD160617_20210707_000336_quickLookExp.fits':(1,1,350.,1700.),\
          'holo4_003_empty_HD160617_20210707_000337_quickLookExp.fits':(1,1,300.,1600.)
                    
         }
else:
        
    shift=4000
        
    info={\

          # with red filter
          'holo4_003_RG610_HD160617_20210707_000234_quickLookExp.fits':(1,1,600.,2000),\
          'holo4_003_RG610_HD160617_20210707_000235_quickLookExp.fits':(1,1,600.,2000),\
          'holo4_003_RG610_HD160617_20210707_000236_quickLookExp.fits':(1,1,600.,shift-2100),\
          'holo4_003_RG610_HD160617_20210707_000237_quickLookExp.fits':(1,1,600.,shift-2100),\
          'holo4_003_RG610_HD160617_20210707_000238_quickLookExp.fits':(1,1,600.,shift-2100),\
          'holo4_003_RG610_HD160617_20210707_000239_quickLookExp.fits':(1,1,600.,shift-2100),\
          'holo4_003_RG610_HD160617_20210707_000240_quickLookExp.fits':(1,1,600.,shift-2100),\
          'holo4_003_RG610_HD160617_20210707_000241_quickLookExp.fits':(1,1,600.,shift-2100),\
          'holo4_003_RG610_HD160617_20210707_000242_quickLookExp.fits':(1,1,600.,shift-2100),\
          'holo4_003_RG610_HD160617_20210707_000243_quickLookExp.fits':(1,1,600.,shift-2100),\
          'holo4_003_RG610_HD160617_20210707_000244_quickLookExp.fits':(1,1,1300.,shift-700),\
          'holo4_003_RG610_HD160617_20210707_000245_quickLookExp.fits':(1,1,250.,shift-1800),\
          'holo4_003_RG610_HD160617_20210707_000246_quickLookExp.fits':(1,1,250.,shift-1800),\
          'holo4_003_RG610_HD160617_20210707_000247_quickLookExp.fits':(1,1,250.,shift-1800),\
      


            # wide scan index 14 ....  21...
          'holo4_003_empty_HD160617_20210707_000248_quickLookExp.fits':(1,1,1400.,shift-700.),\
          'holo4_003_empty_HD160617_20210707_000249_quickLookExp.fits':(1,1,1400.,shift-700.),\
          'holo4_003_empty_HD160617_20210707_000250_quickLookExp.fits':(1,1,400.,shift-1800.),\
          'holo4_003_empty_HD160617_20210707_000251_quickLookExp.fits':(1,1,400.,shift-1800.),\
          'holo4_003_empty_HD160617_20210707_000252_quickLookExp.fits':(1,1,400.,shift-1800.),\
          'holo4_003_empty_HD160617_20210707_000253_quickLookExp.fits':(1,1,0.,shift-800.),\
          'holo4_003_empty_HD160617_20210707_000254_quickLookExp.fits':(1,1,0.,shift-800.),\
          'holo4_003_empty_HD160617_20210707_000255_quickLookExp.fits':(1,1,0.,shift-800.),\
          'holo4_003_empty_HD160617_20210707_000284_quickLookExp.fits':(1,1,300.,500.),\
          'holo4_003_empty_HD160617_20210707_000285_quickLookExp.fits':(1,1,300.,1100.),\
          'holo4_003_empty_HD160617_20210707_000286_quickLookExp.fits':(1,1,200.,1700.),\
          'holo4_003_empty_HD160617_20210707_000287_quickLookExp.fits':(1,1,300.,2300.),\
          'holo4_003_empty_HD160617_20210707_000288_quickLookExp.fits':(1,1,200.,2900.),\
          'holo4_003_empty_HD160617_20210707_000289_quickLookExp.fits':(1,1,200.,3500.),\
          'holo4_003_empty_HD160617_20210707_000290_quickLookExp.fits':(1,1,300.,2200.),\
      

            # narrrow scan : index 76 .....124
          'holo4_003_empty_HD160617_20210707_000317_quickLookExp.fits':(1,1,500.,shift-2100.),\
          'holo4_003_empty_HD160617_20210707_000318_quickLookExp.fits':(1,1,500.,shift-2100.),\
          'holo4_003_empty_HD160617_20210707_000319_quickLookExp.fits':(1,1,500.,shift-1800.),\
          'holo4_003_empty_HD160617_20210707_000320_quickLookExp.fits':(1,1,500.,shift-1800.),\
          'holo4_003_empty_HD160617_20210707_000321_quickLookExp.fits':(1,1,500.,shift-1700.),\
          'holo4_003_empty_HD160617_20210707_000322_quickLookExp.fits':(1,1,500.,shift-1600.),\
          'holo4_003_empty_HD160617_20210707_000323_quickLookExp.fits':(1,1,500.,shift-1500.),\
          'holo4_003_empty_HD160617_20210707_000324_quickLookExp.fits':(1,1,400.,shift-2200.),\
          'holo4_003_empty_HD160617_20210707_000325_quickLookExp.fits':(1,1,400.,shift-2100.),\
          'holo4_003_empty_HD160617_20210707_000326_quickLookExp.fits':(1,1,400.,2200.),\
          'holo4_003_empty_HD160617_20210707_000327_quickLookExp.fits':(1,1,400.,2300.),\
          'holo4_003_empty_HD160617_20210707_000328_quickLookExp.fits':(1,1,400.,shift-1750.),\
          'holo4_003_empty_HD160617_20210707_000329_quickLookExp.fits':(1,1,400.,2400.),\
          'holo4_003_empty_HD160617_20210707_000330_quickLookExp.fits':(1,1,400.,shift-1600.),\
          'holo4_003_empty_HD160617_20210707_000331_quickLookExp.fits':(1,1,300.,shift-2200.),\
          'holo4_003_empty_HD160617_20210707_000332_quickLookExp.fits':(1,1,300.,2000.),\
          'holo4_003_empty_HD160617_20210707_000333_quickLookExp.fits':(1,1,300.,2100.),\
          'holo4_003_empty_HD160617_20210707_000334_quickLookExp.fits':(1,1,350.,shift-1800.),\
          'holo4_003_empty_HD160617_20210707_000335_quickLookExp.fits':(1,1,350.,shift-1750.),\
          'holo4_003_empty_HD160617_20210707_000336_quickLookExp.fits':(1,1,350.,shift-1700.),\
          'holo4_003_empty_HD160617_20210707_000337_quickLookExp.fits':(1,1,300.,2500),\
          'holo4_003_empty_HD160617_20210707_000338_quickLookExp.fits':(1,1,200.,1800),\
          'holo4_003_empty_HD160617_20210707_000339_quickLookExp.fits':(1,1,200.,1800),\
          'holo4_003_empty_HD160617_20210707_000340_quickLookExp.fits':(1,1,200.,2100),\
          'holo4_003_empty_HD160617_20210707_000341_quickLookExp.fits':(1,1,200.,2200),\
          'holo4_003_empty_HD160617_20210707_000342_quickLookExp.fits':(1,1,200.,2200),\
          'holo4_003_empty_HD160617_20210707_000343_quickLookExp.fits':(1,1,200.,2200),\
          'holo4_003_empty_HD160617_20210707_000344_quickLookExp.fits':(1,1,200.,2500),\
          'holo4_003_empty_HD160617_20210707_000345_quickLookExp.fits':(1,1,100.,1800),\
          'holo4_003_empty_HD160617_20210707_000346_quickLookExp.fits':(1,1,100.,1900),\
          'holo4_003_empty_HD160617_20210707_000347_quickLookExp.fits':(1,1,100.,2100),\
          'holo4_003_empty_HD160617_20210707_000348_quickLookExp.fits':(1,1,120.,2100),\
          'holo4_003_empty_HD160617_20210707_000349_quickLookExp.fits':(1,1,200.,2200),\
          'holo4_003_empty_HD160617_20210707_000350_quickLookExp.fits':(1,1,150.,2300),\
          'holo4_003_empty_HD160617_20210707_000351_quickLookExp.fits':(1,1,150.,2400),\
          'holo4_003_empty_HD160617_20210707_000352_quickLookExp.fits':(1,1,50.,2300),\
          'holo4_003_empty_HD160617_20210707_000353_quickLookExp.fits':(1,1,50.,1800),\
          'holo4_003_empty_HD160617_20210707_000354_quickLookExp.fits':(1,1,50.,1800),\
          'holo4_003_empty_HD160617_20210707_000355_quickLookExp.fits':(1,1,50.,2100),\
          'holo4_003_empty_HD160617_20210707_000356_quickLookExp.fits':(1,1,50.,2200),\
          'holo4_003_empty_HD160617_20210707_000357_quickLookExp.fits':(1,1,50.,2300),\
          'holo4_003_empty_HD160617_20210707_000358_quickLookExp.fits':(1,1,50.,2400),\
          'holo4_003_empty_HD160617_20210707_000359_quickLookExp.fits':(1,1,0.,1700),\
          'holo4_003_empty_HD160617_20210707_000360_quickLookExp.fits':(1,1,0.,1800),\
          'holo4_003_empty_HD160617_20210707_000361_quickLookExp.fits':(1,1,0.,1900),\
          'holo4_003_empty_HD160617_20210707_000362_quickLookExp.fits':(1,1,0.,2100),\
          'holo4_003_empty_HD160617_20210707_000363_quickLookExp.fits':(1,1,0.,2200),\
          'holo4_003_empty_HD160617_20210707_000364_quickLookExp.fits':(1,1,0.,2300),\
          'holo4_003_empty_HD160617_20210707_000365_quickLookExp.fits':(1,1,0.,2300),\
         }
        
        
        
        
        
        
 

# Imports

In [50]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [51]:
from astropy.io import fits

In [52]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

In [53]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [54]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [55]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [56]:
dir_logbooks="logbooks"

In [57]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [58]:
FLAG_HOLO=True
home=os.getenv("HOME")

In [59]:
if FLAG_HOLO:
    image_path="holo"
    logbook_csv=os.path.join(dir_logbooks,"auxtelholologbook_2021_07_07_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelholologbook_2021_07_07_" + version + ".xlsx")
else:
    image_path="ronchi170lpmm"
    logbook_csv=os.path.join(dir_logbooks,"auxtelronchilogbook_2021_07_07_" + version + ".csv")
    logbook_xlsx=os.path.join(dir_logbooks,"auxtelronchilogbook_2021_07_07_" + version + ".xlsx")

# Path

In [60]:
HOSTCC=False

In [61]:
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    #path_images=os.path.join(path_auxtel,"data/2021/holo/localreduced/2021-07-06")
    path_images=os.path.join(path_auxtel,"data/2021/holo/quickLookExp/2021-07-07")
else:
    path_auxtel="/Users/dagoret/DATA/AuxTelData2021"
    path_spectractor=os.path.join(path_auxtel,"/users/dagoret/softs/github/lsst/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    #path_images=os.path.join(path_auxtel,"data/2021/holo/localreduced/2021-07-06")
    path_images=os.path.join(path_auxtel,"holo/quickLookExp/2021-07-07")

In [62]:
dir_images=path_images

In [63]:
all_files=os.listdir(dir_images)

In [64]:
all_files

['holo4_003_empty_HD160617_20210707_000334_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000352_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000266_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000300_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000328_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000282_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000252_quickLookExp.fits',
 'holo4_003_RG610_HD160617_20210707_000234_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000331_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000263_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000319_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000357_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000305_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000257_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000363_quickLookExp.fits',
 'holo4_003_empty_HD160617_20210707_000287_quickLookExp

In [65]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [66]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [67]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('^holo4_003_.*_(.*)_quickLookExp[.]fits$',filename)
    
    return m[0]

In [68]:
file_tag_forsorting('holo4_003_empty_HD115169_20210608_000291_quickLookExp.fits')

'000291'

In [69]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['000334', '000352', '000266', '000300', '000328', '000282',
       '000252', '000234', '000331', '000263', '000319', '000357',
       '000305', '000257', '000363', '000287', '000335', '000353',
       '000267', '000301', '000329', '000283', '000235', '000253',
       '000330', '000262', '000356', '000318', '000304', '000256',
       '000362', '000286', '000336', '000350', '000264', '000278',
       '000302', '000280', '000364', '000236', '000250', '000299',
       '000333', '000249', '000261', '000355', '000349', '000255',
       '000285', '000307', '000361', '000337', '000351', '000265',
       '000279', '000303', '000281', '000251', '000237', '000298',
       '000332', '000248', '000260', '000354', '000348', '000254',
       '000284', '000360', '000306', '000269', '000241', '000291',
       '000327', '000341', '000275', '000358', '000244', '000294',
       '000322', '000288', '000258', '000270', '000344', '000240',
       '000268', '000290', '000326', '000340', '000274', '0003

In [70]:
indexes=np.argsort(all_tags)

In [71]:
sorted_files=all_selected_files[indexes]

In [72]:
sorted_files

array(['holo4_003_RG610_HD160617_20210707_000234_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000235_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000236_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000237_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000238_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000239_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000240_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000241_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000242_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000243_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000244_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000245_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000246_quickLookExp.fits',
       'holo4_003_RG610_HD160617_20210707_000247_quickLookExp.fits',
       'holo4_003_empty_HD160617_2

In [73]:
N=len(sorted_files)

### Filter bad files

In [74]:
idx=0
indexes_bad=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header
    #print(hdr0['OBJECT'],"\t",filename)
    if hdr0['OBJECT'] == '35 Tau' or hdr0['OBJECT'] == 'Park position':
        print('bad file ', filename)
        indexes_bad.append(idx)
        
    idx+=1

In [75]:
indexes_bad

[]

### Remove bad files

In [76]:
sorted_files=np.delete(sorted_files,indexes_bad)

### Check for target

In [77]:
re.findall('^holo4_003_.*_(.*)_[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_.*_quickLookExp[.]fits$','holo4_003_RG610_HD120144_20210706_000098_quickLookExp.fits')

['HD120144']

In [78]:
def file_target(filename):
    m=re.findall('^holo4_003_.*_(.*)_[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]_.*_quickLookExp[.]fits$',filename)
    if len(m)>0:
        return m
    elif re.search('NGC4755',filename):
        return ['NGC4755']
    else:
        return []

In [79]:
for filename in sorted_files:
    print(file_target(filename),"\t",filename)

['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000234_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000235_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000236_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000237_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000238_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000239_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000240_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000241_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000242_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000243_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000244_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000245_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD160617_20210707_000246_quickLookExp.fits
['HD160617'] 	 holo4_003_RG610_HD16061

In [80]:
idx=0
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header
    #print(hdr0,"\t",filename)
        
    idx+=1

# Build the logbook from the headers

In [81]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=file_target(filename)[0]
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [82]:
df=pd.DataFrame()

In [83]:
len(all_airmass)

124

In [84]:
N=len(all_indexes)

## Fill with container info

In [85]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [86]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [87]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [88]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [89]:
pd.set_option('display.max_rows', None)

In [90]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2021-07-08T02:50:31.569,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000234_quick...,234,HD160617,RG610,holo4_003,1.022300,5.0,...,12.1,6.0,3.8,600.0,2000.0,NaN,NaN,,1.0,1.0
1,1,2021-07-08T02:51:12.101,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000235_quick...,235,HD160617,RG610,holo4_003,1.022020,5.0,...,12.1,6.0,4.4,600.0,2000.0,NaN,NaN,,1.0,1.0
2,2,2021-07-08T02:51:21.256,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000236_quick...,236,HD160617,RG610,holo4_003,1.021958,5.0,...,12.1,6.0,4.4,600.0,1900.0,NaN,NaN,,1.0,1.0
3,3,2021-07-08T02:51:30.934,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000237_quick...,237,HD160617,RG610,holo4_003,1.021893,5.0,...,12.1,6.0,4.4,600.0,1900.0,NaN,NaN,,1.0,1.0
4,4,2021-07-08T02:51:40.885,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000238_quick...,238,HD160617,RG610,holo4_003,1.021826,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
5,5,2021-07-08T02:51:50.381,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000239_quick...,239,HD160617,RG610,holo4_003,1.021763,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
6,6,2021-07-08T02:51:59.759,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000240_quick...,240,HD160617,RG610,holo4_003,1.021700,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
7,7,2021-07-08T02:52:10.113,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000241_quick...,241,HD160617,RG610,holo4_003,1.021631,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
8,8,2021-07-08T02:52:19.262,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000242_quick...,242,HD160617,RG610,holo4_003,1.021572,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0
9,9,2021-07-08T02:52:29.022,/Users/dagoret/DATA/AuxTelData2021/holo/quickL...,holo4_003_RG610_HD160617_20210707_000243_quick...,243,HD160617,RG610,holo4_003,1.021508,5.0,...,12.1,6.0,4.9,600.0,1900.0,NaN,NaN,,1.0,1.0


# Make logbook


In [91]:
df.to_excel(logbook_xlsx)
df.to_csv(logbook_csv)

In [92]:
logbook_csv

'logbooks/auxtelholologbook_2021_07_07_v4.0.csv'